### Go clustering using david result

In [2]:
import pandas as pd

In [30]:
df = pd.read_csv("../result/all_beta_normalized_result_test_group_rank_tss.csv", index_col=0)
df.drop_duplicates(subset=['gene'], inplace=True)
gene_list = df['gene'].tolist()

In [31]:
# save gene list as txt
with open("../result/gene_listo.txt", "w") as f:
    for gene in gene_list:
        f.write(gene + "\n")

In [38]:
dg = pd.read_csv('../result/dg.txt', sep='\t')
dg.head()

,OFFICIAL_GENE_SYMBOL,Name,Species
0,RPL22,ribosomal protein L22(RPL22),Homo sapiens
1,A2M,alpha-2-macroglobulin(A2M),Homo sapiens
2,MAG,myelin associated glycoprotein(MAG),Homo sapiens
3,TTYH1,tweety family member 1(TTYH1),Homo sapiens
4,NAT8L,N-acetyltransferase 8 like(NAT8L),Homo sapiens


In [37]:
difference = set(gene_list) - set(dg['OFFICIAL_GENE_SYMBOL'].tolist())
difference

{'C20orf85', 'CNTD2', 'METTL7A', 'MRVI1', 'RACGAP1P'}

```DAVID can't find these, change to official symbol manually
C20orf85 -> CIMIP1
CNTD2 -> CCNP
METTL7A -> TMT1A
MRVI1 -> IRAG1
RACGAP1P -> RACGAP1P
```

In [4]:
go_david = pd.read_csv("../result/func_atto_table.txt", index_col=0, sep="\t")

In [5]:
go_david = go_david[['Gene Name','GOTERM_BP_DIRECT','GOTERM_CC_DIRECT','GOTERM_MF_DIRECT']]
go_david.reset_index(inplace=True)
go_david.head()

,ID,Gene Name,GOTERM_BP_DIRECT,GOTERM_CC_DIRECT,GOTERM_MF_DIRECT
0,GBE1,"1,4-alpha-glucan branching enzyme 1(GBE1)","GO:0005975~carbohydrate metabolic process,GO:0...","GO:0005737~cytoplasm,GO:0005829~cytosol,GO:007...","GO:0003844~1,4-alpha-glucan branching enzyme a..."
1,ACCS,1-aminocyclopropane-1-carboxylate synthase hom...,GO:0006520~cellular amino acid metabolic proce...,NaN,"GO:0003824~catalytic activity,GO:0005515~prote..."
2,HPD,4-hydroxyphenylpyruvate dioxygenase(HPD),"GO:0006559~L-phenylalanine catabolic process,G...","GO:0000139~Golgi membrane,GO:0005789~endoplasm...",GO:0003868~4-hydroxyphenylpyruvate dioxygenase...
3,NT5E,5'-nucleotidase ecto(NT5E),"GO:0006196~AMP catabolic process,GO:0006259~DN...","GO:0005654~nucleoplasm,GO:0005829~cytosol,GO:0...","GO:0000166~nucleotide binding,GO:0002953~5'-de..."
4,NT5C1A,"5'-nucleotidase, cytosolic IA(NT5C1A)","GO:0000255~allantoin metabolic process,GO:0006...","GO:0005737~cytoplasm,GO:0005829~cytosol,","GO:0000166~nucleotide binding,GO:0000287~magne..."


In [6]:
go_david.iloc[0,4]
# this fuck boy so i can't just split(",")

'GO:0003844~1,4-alpha-glucan branching enzyme activity,GO:0004553~hydrolase activity, hydrolyzing O-glycosyl compounds,GO:0005515~protein binding,GO:0030246~carbohydrate binding,GO:0043169~cation binding,'

In [7]:
# example data
data = {
    'Gene Name': ['GBE1'],
    'GOTERM_BP_DIRECT': ['GO:0005975~carbohydrate metabolic process,GO:0016070~RNA metabolic process'],
    'GOTERM_CC_DIRECT': ['GO:0005737~cytoplasm,GO:0005829~cytosol'],
    'GOTERM_MF_DIRECT': ['GO:0003844~1,4-alpha-glucan branching enzyme activity']
}
df = pd.DataFrame(data)
df

,Gene Name,GOTERM_BP_DIRECT,GOTERM_CC_DIRECT,GOTERM_MF_DIRECT
0,GBE1,"GO:0005975~carbohydrate metabolic process,GO:0...","GO:0005737~cytoplasm,GO:0005829~cytosol","GO:0003844~1,4-alpha-glucan branching enzyme a..."


In [8]:
import re

separated_df = pd.DataFrame(columns=['ID', 'GO', 'GO_term_root'])

for _, row in go_david.iterrows():

    id = row['ID']
    if type(row['GOTERM_BP_DIRECT']) == float:
        bp = []
    else:
        bp = re.findall(r'GO:.*?(?=~)', row['GOTERM_BP_DIRECT'])
        
    if type(row['GOTERM_CC_DIRECT']) == float:
        cc = []
    else:
        cc = re.findall(r'GO:.*?(?=~)', row['GOTERM_CC_DIRECT'])
    
    if type(row['GOTERM_MF_DIRECT']) == float:
        mf = []
    else:
        mf = re.findall(r'GO:.*?(?=~)', row['GOTERM_MF_DIRECT'])
    go_terms = {
        'GOTERM_MF_DIRECT': bp,
        'GOTERM_CC_DIRECT': cc,
        'GOTERM_MF_DIRECT': mf
    }
    for go_term_root, go_terms_list in go_terms.items():
        for go_term in go_terms_list:
            separated_df = pd.concat([separated_df, pd.DataFrame([[id, go_term, go_term_root]], columns=['ID', 'GO', 'GO_term_root'])], ignore_index=True)
separated_df

,ID,GO,GO_term_root
0,GBE1,GO:0003844,GOTERM_MF_DIRECT
1,GBE1,GO:0004553,GOTERM_MF_DIRECT
2,GBE1,GO:0005515,GOTERM_MF_DIRECT
3,GBE1,GO:0030246,GOTERM_MF_DIRECT
4,GBE1,GO:0043169,GOTERM_MF_DIRECT
...,...,...,...
10973,ZFPM2,GO:0000785,GOTERM_CC_DIRECT
10974,ZFPM2,GO:0001673,GOTERM_CC_DIRECT
10975,ZFPM2,GO:0005634,GOTERM_CC_DIRECT
10976,ZFPM2,GO:0005654,GOTERM_CC_DIRECT


In [9]:
separated_df

,ID,GO,GO_term_root
0,GBE1,GO:0003844,GOTERM_MF_DIRECT
1,GBE1,GO:0004553,GOTERM_MF_DIRECT
2,GBE1,GO:0005515,GOTERM_MF_DIRECT
3,GBE1,GO:0030246,GOTERM_MF_DIRECT
4,GBE1,GO:0043169,GOTERM_MF_DIRECT
...,...,...,...
10973,ZFPM2,GO:0000785,GOTERM_CC_DIRECT
10974,ZFPM2,GO:0001673,GOTERM_CC_DIRECT
10975,ZFPM2,GO:0005634,GOTERM_CC_DIRECT
10976,ZFPM2,GO:0005654,GOTERM_CC_DIRECT


In [10]:
separated_df.to_csv("../result/separated_go_terms.csv")